<a href="https://colab.research.google.com/github/taberdar/colab/blob/main/Asymmetric_key_signing_at_end.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Asymmetric key demonstration



# Run this first

In [ ]:
import rsa
def asymmetric_encrypt(message, text_public_key):
  public_key = rsa.PublicKey.load_pkcs1(text_public_key)
  return rsa.encrypt(message.encode('utf8'), public_key).hex()

def asymmetric_decrypt(messageinhex, private_key):
  return rsa.decrypt(bytes.fromhex(messageinhex), private_key).decode('utf8')

def asymmetric_sign(message, private_key):
  return rsa.sign(message.encode('utf8'), private_key, 'SHA-1').hex()

def asymmetric_check(message, signatureinhex, text_public_key):
  public_key = rsa.PublicKey.load_pkcs1(text_public_key)
  return rsa.verify(message.encode('utf8'), bytes.fromhex(signatureinhex), public_key)

def make_public_key(publickey):
  ff=publickey.save_pkcs1()
  return(ff.decode('utf8'))

def read_public_text_key(publickeytext):
  key=rsa.PublicKey.load_pkcs1(publickeytext.encode('utf8'))
  return(key)

def generate_keys(keysize):
  (public, private) = rsa.newkeys(keysize)
  publictext = public.save_pkcs1().decode('utf8')
  return publictext, public, private
  
keysize = 512  # In real life we would use a keysize of more than 2048

# Try it yourself

## Publish your public keys

In [ ]:
(my_pub_text, my_public_key, my_private_key) = generate_keys(keysize)
# announce public keys to Google Classroom 
# Copy the text below and paste it into the Google Classroom announcements
print(my_pub_text)

# Copy a key from Google Classroom

In [ ]:
# Copy a public key from Google Classroom
# and put it into a variable called key
key = """
-----BEGIN RSA PUBLIC KEY-----
MEgCQQCZVcDe/wa9pXgifK0yI8zrrjQz4iTdv8VYu81bI6rV8NCuZ3Hjj6G3qjYY
8M1P8EbLebyvEoHXjURBGzIPSbmHAgMBAAE=
-----END RSA PUBLIC KEY-----
"""
print(key)

In [ ]:
# Now we can encrypt messages for that person with the public key
message = 'A message for the person with the public key'
message_for_person_with_public_key = asymmetric_encrypt(message, key)
print(message_for_person_with_public_key)

## Decrypt a message for you

In [ ]:
# Wait for someone to send you a message encrypted with your public key
# it will look a bit like this - copy it from Google Classroom - you will need to do this.
encrypted_message_for_you = '19cec02c294ee7cd44f57701928cf5e30bb26dea516545939051b36bc64e45a1549e3fe8d0c925241e6f665ec3c2d508f06b5fbcb8058dee4e1b38afceb57c9b'

message = asymmetric_decrypt(encrypted_message_for_you, my_private_key)
print(message)

# Full worked example for reference

## Generate some keys

In [ ]:
# Generate some keys
# It's always Alice, Bob and Charlie in the crypto world.
(alice_pub_text, alice_pub, alice_priv) = generate_keys(keysize)
(bob_pub_text, bob_pub, bob_priv) = generate_keys(keysize)
(charlie_pub_text, charlie_pub, charlie_priv) = generate_keys(keysize)
print("Alice's public key")
print(alice_pub_text)
print()
print("Bob's public key")
print(bob_pub_text)
print()
print("Charlie's public key")
print(charlie_pub_text)


## Send a message for Bob alone





In [ ]:
# We want to send a message to Bob
message = 'hello Bob!'
print(message)

In [ ]:
# We encrypt with Bob's public key
messageForBobEncrypted = asymmetric_encrypt(message, bob_pub_text)
messageForBobEncrypted

In [ ]:
# Bob decrypts with his private key
messageForBobDecrypted = asymmetric_decrypt(messageForBobEncrypted, bob_priv)
print(messageForBobDecrypted)

## Charlie can't read the message for Bob

In [ ]:
# Charlie can't use her key to decrypt
asymmetric_decrypt(messageForBobEncrypted, charlie_priv)

## The message can't be tampered with

In [ ]:
#If we tamper with the message, we get an error
tampered = '00'+ messageForBobEncrypted[2:]
print(tampered)
asymmetric_decrypt(tampered, bob_priv)

# Signing messages (optional)

In [ ]:
# We can sign messages so that the recipient knows it was from the sender
# So Alice is sending something that she is asserting was sent by her
# She uses her private key to do this
messagefromalice = 'Message from Alice'
signature = asymmetric_sign(messagefromalice, alice_priv)
print(messagefromalice)
print("The signature of the message is:", signature)

Message from Alice
The signature of the message is: 49b9c42a8fabea1a8c02e770d6460b8634eaba6f66e576727de251951bbf96fdc1323c27602dd99b8a9370514ce1ab7e90f27c001da884a20797bc31f2c94bab


In [ ]:
# We can confirm that Alice created a signature of the message
# by using Alice's public key
print(messagefromalice)
'OK' if asymmetric_check(messagefromalice, signature, alice_pub_text) == 'SHA-1' else 'Not OK!'

Message from Alice


'OK'

In [ ]:
# If the message is tampered with then the signature check fails
messagefromalice = "This wasn't from Alice"
print(messagefromalice)
asymmetric_check(messagefromalice, signature, alice_pub_text)

This wasn't from Alice


VerificationError: ignored

In [ ]:
# Normally we would sign a message then encrypt it
# with the public key of the intended recipient
# and we would send both to the intended recipient
mm = 'New secret message definitely from Alice'
sign = asymmetric_sign(mm, alice_priv)
ee = asymmetric_encrypt(mm, bob_pub_text)

print(mm)
print(sign)
print(ee)

New secret message definitely from Alice
057999ff1ec509d2361c3aef41964792c80082785cffdca20866411995680ffe6c297594f3471099c57ecb384aa0aef0edb9b7a68c4561e519da0ba1ce536d99
93f1bef01a34dda87b37d57a9643c5269378a121ecff33392e0d882617a32fa2604ecbc910d094de277eae24677a7db18c167a689c99a96ca3221fd2b0c9e020


In [ ]:
# The recipient would decrypt it the message
# then check the signature
dd = asymmetric_decrypt(ee, bob_priv)
fromalice = asymmetric_check(dd, sign, alice_pub_text)
print(dd)
'Confirmed from Alice' if fromalice == 'SHA-1' else 'Not from Alice!'

New secret message definitely from Alice


'Confirmed from Alice'